In [1]:
import pandas as pd
import sys
sys.path
sys.path.append('../models')
import numpy as np
import pickle
import random
import os
import math
import sim_functions_cleaner  as sf
import operator
import study
import time
import TS_personal_params_pooled as pp
import TS_global_params_pooled as gtp
from numpy.random import uniform

#sys.path.append('../simulation')
import TS_fancy_pooled 
#import TS_fancy_pooled 
import eta
import pooling_bandits as pb
import warnings 
warnings.simplefilter('ignore')

In [2]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
import logging
logging.getLogger("FFC").setLevel(logging.ERROR)

In [3]:
##for each person generate dates
##easiest to take from original data, this might best mimic actual situation 

In [4]:
#root =  '../../../../Volumes/dav/HeartSteps/pooling_rl_shared_data/processed/'

In [5]:
#experiment.population[1]

In [6]:
def initialize_policy_params_TS(experiment,context_dimension):
    
    global_p =gtp.TS_global_params(10,context_dimension) 
    personal_p = pp.TS_personal_params()
    
    #print(type(personal_p))
    
    for person in experiment.population.keys():
        initial_context = [0 for i in range(context_dimension)]
        personal_p.mus0[person]= global_p.get_mu0(initial_context)
        personal_p.mus1[person]= global_p.get_mu1(initial_context)
        personal_p.mus2[person]= global_p.get_mu2(initial_context)

        personal_p.sigmas0[person]= global_p.get_asigma(len( personal_p.mus0[person]))
        personal_p.sigmas1[person]= global_p.get_asigma(len( personal_p.mus1[person]))
        personal_p.sigmas2[person]= global_p.get_asigma(len( personal_p.mus2[person]))
     
        
        
        personal_p.batch[person]=[[] for i in range(len(experiment.person_to_time[person]))]
        personal_p.batch_index[person]=0
        
        personal_p.etas[person]=eta.eta()
   
        personal_p.last_update[person]=experiment.person_to_time[person][0]
        
        
    return global_p ,personal_p     

In [7]:
#days = pd.date_range(start = '7/22/2015',end =pd.Timestamp('7/22/2015')+pd.DateOffset(days=45),freq='30T')

In [8]:
#ft = experiment.study_days[96]

In [9]:
#yesterday = ft-pd.DateOffset(days=1)
#end = yesterday.replace(hour=0,minute=0)
#start = experiment.study_days[0]
#end

In [10]:
def new_kind_of_simulation(experiment,policy=None,personal_policy_params=None,global_policy_params=None):
    

    for time in experiment.study_days:
            if time> experiment.study_days[0]:
                history  = pb.make_history(experiment)
            if time==experiment.last_update_day+pd.DateOffset(days=1):
                experiment.last_update_day=time
                ##global update
                #print(time)
                #print(experiment.last_update_day+pd.DateOffset(days=1))
                ##am i checking the current time (need to check the 
                #current time make sure i'm not using all of the history)
               
                #print(history)
                temp_params = TS_fancy_pooled.global_updates(history[0],history[1],train_type = 'Static')
                print(temp_params['cov'].shape)
                global_policy_params.update_params(temp_params)
                del temp_params
                ##update global params using these temp_params
                
                
                
            ##update global context
            ##global context shared across all participants
            tod = sf.get_time_of_day(time)
            dow = sf.get_day_of_week(time)
            if time==experiment.study_days[0]:
                
                weather = sf.get_weather_prior(tod,time.month)
            elif time.hour in experiment.weather_update_hours and time.minute==0:
                weather = sf.get_next_weather(str(tod),str(time.month),weather)
            ##location depends on person 
            
            for person in experiment.dates_to_people[time]:
                dt=False
                action = 0 
                prob=0
                #1
                ##for every active person update person specific aspects of their context
                participant = experiment.population[person]
                #update global context variables
                participant.set_tod(tod)
                participant.set_dow(dow)
                participant.set_wea(weather)
                
                
                availability = (uniform() < 0.8)
                participant.set_available(availability)
                
                if time == participant.times[0]:
                    #get first location 
                    location = sf.get_location_prior(str(participant.gid),str(dow),str(tod))
                    participant.set_inaction_duration(0)
                    participant.set_action_duration(0)
                    participant.set_duration(0)
                    participant.set_dosage(0)
                    #personal_policy_params.etas[participant.pid]
                    
                    
                if time <= participant.times[1]:
                    steps_last_time_period = 0  
                    
                    ##set first pre-treatment, yesterday step count, variation and dosage
                else:
                    #print(time)
                    steps_last_time_period = participant.find_last_time_period_steps(time)
                    participant.update_duration(steps_last_time_period)
             
                    #get var id
                    
                if time.date() <= participant.times[0].date():
                    steps_yesterday = 0    
                else:
                    steps_yesterday =  participant.find_yesterday_steps(time)
                    steps_yesterday = sf.to_yid(steps_yesterday)
                    
                if time.hour in experiment.location_update_hours:
                    location = sf.get_next_location(participant.gid,dow,tod,participant.get_loc())
                
                if time.date()>(participant.times[0]+pd.DateOffset(days=1)).date():
                  
                    if time.hour==0 and time.minute==0:
                        variation = participant.find_variation(time)
                else:
                    variation = 1 
                
                participant.set_loc(location)
                ##maybe faster to update instead of query?
                participant.set_last_time_period_steps(steps_last_time_period)
                participant.set_yesterday_steps(steps_yesterday)
                participant.set_variation(variation)
                
                ##continue
                #2
                ##for every active person take an action according to current context, policy, and parameters
                
                
                ##for now:
                ##eval with empty array 
                if time in participant.decision_times and availability:
                    
                    dt=True
                    action=0
                    
                    if policy==None:
                        action = sf.get_action(policy)
                        
                        
                        
                    elif policy=='TS':
                        #some context slice
                            prob = TS.prob_cal_ts([int(tod),int(dow)],participant.dosage,\
                                              personal_policy_params.mus2[participant.pid],personal_policy_params.sigmas2[participant.pid],\
                                                 global_policy_params)
                            action = int(uniform() < prob)
                            
                        
                            
                    elif policy=='TS_fancy':
                        #previous
                        #Z, X, mu_beta, Sigma_beta, init,current_eta
                        
                        ##need to make eta part of the global policy params
                        prob = TS_fancy_pooled.prob_cal([location,weather,steps_last_time_period,variation,steps_yesterday],participant.dosage,\
                                              personal_policy_params.mus2[participant.pid],personal_policy_params.sigmas2[participant.pid],\
                                                 global_policy_params,personal_policy_params.etas[participant.pid])
                        action = int(uniform() < prob)
                        
                        
                        
                    ##is this the same as in the TS?
                    ##don't think so, but for now keep like this
                    ##no it isn't, i have to redo this
                    participant.update_dosage(action)
                    
                    context = [action,participant.gid,tod,dow,location,weather,participant.get_prekey(participant.duration,steps_last_time_period),\
                              steps_yesterday,variation,sf.dosage_to_dosage_key(participant.dosage)]
                    steps = sf.get_steps_action(context)
                else:
                    steps = sf.get_steps_no_action(participant.gid,tod,dow,location,weather)
                
                
                
                personal_policy_params.update_batch(participant.pid,[0,location,weather,steps_last_time_period,variation,steps_yesterday,participant.dosage,\
                                                                        int(availability) ,action,prob,steps])  
                
                ##history:
                context_dict =  {'steps':steps,'action':action,'weather':weather,'location':location,\
                                'ltps':steps_last_time_period,'duration':participant.duration,\
                                'study_day':participant.current_day_counter,'decision_time':dt}
                participant.history[time]=context_dict
                
            #3
            ##for every active person generate a step count given current context
            
            
            
            
            ##update at midnight (here we have ensured that no one has a ) experiment.update_hour
            
                if time.hour==0 and time.minute==0 :
                    ##update the policy
                    #print(personal_policy_params.batch_index[participant.pid])
               
                    batch = personal_policy_params.batch[participant.pid]\
                    [:personal_policy_params.batch_index[participant.pid]]
                    
                    
                    
                    ##do global updates
                    
                    if time==personal_policy_params.last_update[participant.pid]+pd.DateOffset(days=8):
                        #print('batch')
                        #print(batch)
                        temp = TS_fancy_pooled.policy_update( global_policy_params,batch,\
                                              personal_policy_params.mus0[participant.pid],\
                                               personal_policy_params.sigmas0[participant.pid],\
                                               personal_policy_params.mus1[participant.pid],\
                                               personal_policy_params.sigmas1[participant.pid],\
                                               personal_policy_params.mus2[participant.pid],\
                                               personal_policy_params.sigmas2[participant.pid],\
                                               proxy=True
                                              )
                    
                      
                
                        eta_params = temp[2]
                        personal_policy_params.etas[participant.pid].update_params(eta_params)
                        personal_policy_params.etas[participant.pid].which_function = 'other'
                        personal_policy_params.last_update[participant.pid]=time
                    else:
                        ##calculate posterior
                        
                        ##pretend they are scaled
                        if time>participant.times[0]:
                           
                            temp = pb.calculate_posterior(global_policy_params,\
                                                      participant.pid,participant.current_day_counter,\
                                                          history[0], history[1] )
                        
                        #temp = TS_fancy_pooled.policy_update( global_policy_params,batch,\
                                           #   personal_policy_params.mus0[participant.pid],\
                                             #  personal_policy_params.sigmas0[participant.pid],\
                                             #  personal_policy_params.mus1[participant.pid],\
                                             #  personal_policy_params.sigmas1[participant.pid],\
                                             #  personal_policy_params.mus2[participant.pid],\
                                             #  personal_policy_params.sigmas2[participant.pid],\
                                           #
                                             # )
                        
                        else:
                            temp = [personal_policy_params.mus2[participant.pid],personal_policy_params.sigmas2[participant.pid]]
                    
                    #print(temp)
                    mu_beta = temp[0]
                    Sigma_beta = temp[1]
                    personal_policy_params.update_mus(participant.pid,mu_beta,2)
                    personal_policy_params.update_sigmas(participant.pid,Sigma_beta,2)
                    



In [11]:
experiment = study.study()

In [ ]:
glob,personal = initialize_policy_params_TS(experiment,5)

In [7]:
i = [0, [0, 1, 0, 1, 0], 0, 1, 0, 0, 0.0]

In [8]:
glob.x_index

6

In [15]:
len([0, 3, 1, 0, 1, 0, 17, 0, 0, 0, 0.0])

11

In [1]:
j.extend(i[1:])

NameError: name 'j' is not defined

In [ ]:
start = time.time()
new_kind_of_simulation(experiment,'TS_fancy',personal,glob)
end = time.time()
print(end-start)

called
(3, 3)


called
(8, 8)


called
(15, 15)


called
(28, 28)


called
(41, 41)


...

called
(2564, 2564)


called
(2605, 2605)


called
(2645, 2645)


called
(2683, 2683)


called


In [ ]:
all_steps  = [
]
for p in experiment.population.values():
    all_steps.extend([v['steps'] for v in p.history.values()])

In [29]:
np.array(all_steps).mean()

269.48361421434157

In [ ]:
[223.62212776908694,]

In [17]:
all_steps  = set([])
for p in experiment.population.values():
    all_steps.update([v['action'] for v in p.history.values()])
all_steps

{0, 1}

In [5]:
def get_today_variance(today,lookup,all_steps):
    morning = today.replace(hour=0,minute=0)
    
    tonight = today.replace(hour=23,minute=30)
    
    steps = all_steps[lookup[morning]:lookup[tonight]+1]
    
   
    
    steps = transform_to_hour(steps)
    
    return np.array(steps).std()

In [6]:
means = []
for n in range(1000):
    gr=sf.simulate_run(1,days,[])
    means.append(np.array(gr[0][0]).mean())

In [49]:
np.array(gr[0][0]).mean()

446.51003316358634

In [7]:
np.array(means).mean()

137.29867338336618

In [38]:
gr[1][0]

[2, 0, 0, 0, 2, 1, 0, 1, 1]

In [13]:
get_today_variance(gr[0],gr[1],gr[2])

False


0.0

In [14]:
def get_all_day_slices(first,last):
    slices = []
    day = first
    
    while day<last:
        slice_one = [day]
        #slice_one.append(day.replace(hour=23,minute=30))
        
        day_two = day+pd.DateOffset(days=1)
        slice_one.append(day_two)
        slices.append(slice_one)
        day=day_two
    return slices

In [8]:
def transform_to_hour(steps):
    to_return = []
    for i in range(0,len(steps)-1,2):
        to_return.append(steps[i]+steps[i+1])
    return to_return

In [35]:
def day_slices_to_median(day_slices,day_lookup,all_steps):
    days_steps = []
    for ds in day_slices:
        steps = all_steps[day_lookup[ds[0]]:day_lookup[ds[1]]]
       
        steps = transform_to_hour(steps)
        print(len(steps))
        days_steps.append(np.array(steps).std())

    return np.median(np.array(days_steps))
        

In [19]:
x=get_all_day_slices(gr[3],gr[4])

In [43]:
gr[4]

Timestamp('2015-07-29 00:00:00', freq='30T')

In [13]:
[1]+[2]+[3,4]

[1, 2, 3, 4]

In [3]:
    last_index = dd[one_week_ago.date()][0]
    
    day_slices=get_all_day_slices(last_index,dd[i.date()][0])
    
    median = day_slices_to_median(day_slices,gr[2],gr[5])

You can pass in a bunch of decision times. I just tested with one. 

Right now I think the number of people is being totally ignored. This needs to be fixed. 

In [12]:
x=sim_functions.simulate_run(num_people,temp_indices,decision_times)

NameError: name 'temp_indices' is not defined

In [14]:
np.array(x).mean()
#np.array(x).std()

207.01370855122792

In [15]:
np.array(x).mean()
np.array(x).std()

267.8619020654599

In [ ]:
2-2-0-1-1-0-0-1

In [5]:
root =  '../../../../Volumes/dav/HeartSteps/pooling_rl_shared_data/temporary/'
data_root =  '../../../../Volumes/dav/HeartSteps/pooling_rl_shared_data/distributions/'


In [9]:
with open('../../../../Volumes/dav/HeartSteps/pooling_rl_shared_data/distributions/steps_both_groups_logs_dosage_estf.pkl'.format(),'rb') as f:
    df = pickle.load(f)

In [10]:
df.keys()

dict_keys(['2-0-2-1-2-1-0-3-18', '2-1-1-1-0-1-0-3-77', '2-0-0-0-3-3-0-3-9', '2-0-2-0-2-2-0-4-37', '2-1-2-1-0-1-0-4-70', '2-0-1-0-2-0-0-4-35', '2-1-2-1-0-1-0-3-13', '2-0-2-2-2-0-0-4-48', '2-0-1-1-2-1-0-3-28', '2-0-0-2-1-1-0-3-8', '2-0-2-0-2-0-0-3', '2-1-1-1-0-2-0-4-78', '2-0-1-1-2-1-0-3-31', '2-0-1-1-1-1-0-4-70', '2-0-1-2-1-3-0-4-66', '2-0-2-1-0-1-0-3-31', '2-0-1-2-0--1-0-4-25', '1-0-2-0-3-0-0-4-16', '2-1-2-0-3-2-0-4-3', '2-0-0-1-0-1-0-3', '2-1-1-1-1-1-0-3-18', '2-1-1-0-1-1-0-4-53', '2-1-2-2-1-1-0-4-35', '2-0-1-2-0--1-0-3-31', '2-0-1-2-0-3-0-4-64', '2-0-2-1-1-2-0', '1-1-1-0-3-3-0-3-38', '2-0-0-1-2-1-0-4-77', '2-0-2-2-1-1-1', '2-0-1-2-1-2-0', '2-0-2-2-0-2-1-3-3', '2-0-1-1-1-1-1-3-9', '1-0-2-0-0-0-0-4-94', '2-0-2-1-1-2-0-3-22', '2-0-1-2-0-0-1-4-9', '2-0-2-0-2-2-0-4-39', '2-1-1-1-2-0-0-4-91', '2-0-2-1-0-2-0-3-21', '2-0-1-0-0-0-0-4-38', '2-0-2-1-0-3-0-3-96', '2-0-2-0-0-1-1-4-1', '2-0-1-1-2-0-0-3-47', '2-0-1-1-1-2-0-4-9', '2-0-1-0-1-1-0-4-30', '2-0-1-1-0-2-0-3-53', '2-1-1-1-0-0-0-4-95', '2-1